In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from datasets import load_dataset



dataset = load_dataset("openlifescienceai/medmcqa")
train_dataset = dataset["train"]
curr_dataset = train_dataset


config = {
    'model_name': 'pubmed_bert',
}
model_full_name = 'bert-base-uncased' if config['model_name'] == "general_bert" else "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
print(f"Using model: {model_full_name}")
tokenizer = BertTokenizer.from_pretrained(model_full_name)


Q = curr_dataset["question"]
A,B,C,D = curr_dataset["opa"], curr_dataset["opb"], curr_dataset["opc"], curr_dataset["opd"]

# if load_hints:
#   hints = None # load from somewhere
#   inputs = map(lambda h,q,a,b,c,d:f"Question: Given that {h}, {q}\nA: {a}\nB: {b}\nC: {c}\nD: {d}\n", hints,Q,A,B,C,D)
#   input_tokens = self.tokenizer(list(inputs), padding=True, truncation=True, return_tensors="pt")

inputs = map(lambda q,a,b,c,d:f"Question: {q}\nA: {a}\nB: {b}\nC: {c}\nD: {d}\n", Q,A,B,C,D)
input_tokens = tokenizer(list(inputs), padding=True, truncation=True, return_tensors="pt")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using model: microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [2]:
explanations = curr_dataset["exp"]
explanations = list(map(lambda e: str(e), explanations))
exp_tokens = tokenizer(explanations, padding=True, truncation=True, max_length=256, return_tensors="pt")

In [3]:
exp_exist = []
for i in curr_dataset["exp"]:
  if i == None:
    exp_exist.append(0)
  else:
    exp_exist.append(1)

In [5]:
import torch
bert_version = config["model_name"]
train_dataset_with_explanations = torch.utils.data.TensorDataset(input_tokens.input_ids, input_tokens.attention_mask,
                                exp_tokens.input_ids, exp_tokens.attention_mask, torch.tensor(exp_exist),
                                                torch.tensor(train_dataset["cop"]).long())

torch.save(train_dataset_with_explanations, 
           '/root/pubmedQA_291/dataset_pickles/medmcqa/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext/classification_style/train_dataset_with_explanations.pt')

: 